In [1]:
'''
Functions for downloading and reading MNIST data.
'''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import tempfile

import numpy
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

mnist = read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
'''
function for initialize params
'''

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return initial

In [3]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [4]:
x = tf.placeholder('float', shape=[None, 784])
y_ = tf.placeholder('float', shape=[None, 10])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [5]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [6]:
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [7]:
keep_prob = tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [8]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [11]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(1, 10001):
        batch = mnist.train.next_batch(50)
        if i % 500 == 0:
            train_accuracy = sess.run(accuracy, feed_dict = {x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy:%g' % (i, train_accuracy))
        sess.run(train_step, feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})
    print("test accuracy %g" % (accuracy.eval(feed_dict={x: mnist.test.images, 
                                                      y_: mnist.test.labels, 
                                                      keep_prob: 1.0})))

step 500, training accuracy:0.9
step 1000, training accuracy:0.96
step 1500, training accuracy:1
step 2000, training accuracy:1
step 2500, training accuracy:1
step 3000, training accuracy:1
step 3500, training accuracy:0.98
step 4000, training accuracy:1
step 4500, training accuracy:1
step 5000, training accuracy:1
step 5500, training accuracy:1
step 6000, training accuracy:1
step 6500, training accuracy:0.98
step 7000, training accuracy:1
step 7500, training accuracy:1
step 8000, training accuracy:1
step 8500, training accuracy:1
step 9000, training accuracy:1
step 9500, training accuracy:1
step 10000, training accuracy:1
test accuracy 0.9906
